In [3]:
!pip install sentence_transformers
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 93.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.5 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=e5648740713a279e77570928bc183fb7b4438dc932f952e25e0e26faabaa3d07
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.1 MB/s 

In [4]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import openai
import requests

In [5]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [6]:
#api 키 에러나서 재발급 받았습니당
openai.api_key = "" #ChatGPT KEY 받아오기
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

In [8]:
#Gdrive Path
train_data = pd.read_csv('/content/gdrive/MyDrive/DB.csv')
train_index = pd.read_csv('/content/gdrive/MyDrive/index.csv')
train_index.head(20)

#NaN 제거하기
#train_data.dropna(axis=1)

,label,s,f
0,불면증,0,10
1,안면홍조,11,22
2,생리불순,23,34
3,갱년기영양제(백수오),35,48
4,배뇨장애+요실금수술,49,63
5,진땀,64,76
6,갱년기 유산균+식이섬유,77,91
7,퇴행성 관절염,92,106
8,골다공증,107,117
9,호르몬치료(HRT),118,135


In [9]:
# Add 'embeddingn' row, 질문의 임베딩값 저장하기
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)

# calculate cosine similarity
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

train_data

# def calculate_distance(text, model):
#     embedding = model.encode(text) #입력받은 text의 임베딩값 구하기
#     train_distance['distance'] = train_distance['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze()) #저장된 임베딩값과 입력받은 임베딩 값의 거리 구해서 distance column에 저장
#     return train_distance

# # 거리가 가장 먼 질문
# def get_max_distance_answer(df):
#     answer = qa_dict[df.loc[df['distance'].idxmax(), 'Q']]
#     return answer

#df = df[~df['챗봇'].isna()] # isna : NaN 제거


,Q,A,label,embedding
0,"원래 잠을 많이 자는 편은 아닌데, 요즘은 거의 못 자서 잠들기가 힘듭니다. \n갱...","갱년기 불면증은 갱년기 여성들이 겪는 수면장애로, 수면의 질과 양이 감소하는 것이 ...",불면증,"[-0.42966908, -0.38425782, 0.5401903, -0.10943..."
1,원인이 무엇인가요?,갱년기 불면증의 원인은 여성 호르몬인 에스트로겐의 분비 감소로 인한 수면의 질과 양...,불면증,"[-0.08331859, -0.4991247, 0.22772601, -0.38741..."
2,수면제 먹지 않고 잘수 있는 방법이 뭘까요?,"갱년기 불면증 완화 방법으로는 적정 체중 유지, 천연 보충제 섭취, 건강한 식단, ...",불면증,"[-0.19923586, -0.18425219, 0.543422, 0.3576777..."
3,수면환경 개선은 어떻게 하나요?,"수면환경 개선이라는 것이 어떤 것을 말씀하시는 건가요? 침대, 방, 조명 등을 말씀...",불면증,"[-0.5954792, 0.14412792, 0.26517367, 0.1422892..."
4,"침대, 방, 조명 같은 거 맞아요.","침대는 편안하게, 방은 시원하게, 조명은 어두운 것이 좋습니다. 또한, 스마트폰, ...",불면증,"[0.2597087, -0.28704673, -0.016775867, -0.2699..."
...,...,...,...,...
677,갱년기 치료 예방법은?,"여성 갱년기의 경우 많은 관심과 적극적인 치료로 완화되는 경우가 많지만, 남성 갱년...",남성갱년기,"[-0.42432237, -0.7201248, 0.093347766, 0.19700..."
678,남성 갱년기의 원인은 무엇인가요?,남성 갱년기의 원인으로는 노화로 인한 뇌와 고환 기능의 저하 및 남성 호르몬 분비 ...,남성갱년기,"[-0.35925066, -0.63448423, 0.21190982, 0.01691..."
679,남성 갱년기 대해 궁금합니다.,갱년기란 성호르몬(sex hormone)의 감소로 인해 여러 증상이 나타나는 기간을...,남성갱년기,"[-0.18914673, -0.937516, 0.11306372, 0.1020882..."
680,남성 갱년기의 진단기준은 무엇입니까?,"폐경 이후 급속히 증상이 나타나는 여성 갱년기와 달리, 남성 갱년기는 서서히 진행되...",남성갱년기,"[-0.3402798, -0.6937028, 0.18714343, -0.244301..."


In [35]:
train_index['embedding'] = train_index.apply(lambda row: model.encode(row.label), axis = 1)
train_index.head(44)

,label,s,f,embedding,score
0,불면증,0,10,"[-0.6531773, 0.04081554, 0.7766159, -0.1107208...",0.353299
1,안면홍조,11,22,"[-0.35724393, -0.5506344, 0.05690285, 0.057393...",0.340380
2,생리불순,23,34,"[-0.87005776, -0.22222981, -0.26471972, 0.2634...",0.368053
3,갱년기영양제(백수오),35,48,"[-0.5289643, -0.39791065, 0.8435944, 0.4298065...",0.363553
4,배뇨장애+요실금수술,49,63,"[-0.39127314, 0.00025672533, -0.11637302, 1.09...",0.130214
5,진땀,64,76,"[-0.2485298, -0.26893857, -0.052861508, -0.632...",0.363044
6,갱년기 유산균+식이섬유,77,91,"[0.026615918, -0.8675919, 0.25804666, 0.709526...",0.467252
7,퇴행성 관절염,92,106,"[-0.531884, 0.3720626, -0.097410284, -0.204592...",0.396046
8,골다공증,107,117,"[0.15695296, -0.536807, -0.047373485, -0.40884...",0.316786
9,호르몬치료(HRT),118,135,"[-0.7304946, -0.2100901, -0.15829049, 0.647662...",0.270539


In [23]:
def return_answer(question) :
    embedding = model.encode(question)
    train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    return train_data.loc[train_data['score'].idxmax()]['A']

In [111]:
def u_index(question) :
    embedding = model.encode(question)
    train_index['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)

    find_index = train_index[train_index['score']>0.7]
    find_data = pd.DataFrame()

    find_data = find_data.append(train_data)
    for s,f in zip(find_index['s'],find_index['f']):
      pd.concat([find_data,train_data.loc[s:f]],axis=0,ignore_index=True)

    find_data['score'] = find_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    return find_data.loc[find_data['score'].idxmax()]['A'],find_data.loc[find_data['score'].idxmax()]['score']

In [116]:
u_index("으라차차")

<ipython-input-111-fc34057b89bd>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  find_data = find_data.append(train_data)


('푸시업은 가슴과 삼두근을 강화하는 운동입니다. 아래와 같이 진행합니다.\n \n 바닥에 엎드립니다.\n 양손을 어깨 너비로 벌립니다.\n 팔꿈치를 굽혀 몸을 내립니다.\n 팔꿈치를 펴서 몸을 올립니다.\n 플랭크는 복근과 전신 근육을 강화하는 운동입니다. 아래와 같이 진행합니다.\n \n 바닥에 엎드립니다.\n 팔꿈치를 굽혀 팔꿈치와 발끝만 바닥에 닿게 합니다.\n 몸을 일직선으로 유지합니다.\n 아래 링크에서 자세한 설명과 함께 동영상으로 확인하실 수 있습니다. :',
 0.3417958)

In [24]:
return_answer("갱년기에 좋은 약이 있을까?")

'갱년기 증상을 완화시키는 약은 여러 가지가 있는데요. 그 중 대표적인 약물은 호르몬 치료제입니다. 호르몬 치료제는 여성 호르몬인 에스트로겐과 프로게스테론을 체외에서 대체로 주입하여 갱년기 증상을 완화시키는 약물입니다. \n \n 1. 에스트로겐 치료제: 에스트로겐의 부족으로 인한 갱년기 증상을 완화시키는 약물입니다. 에스트로겐은 여성의 난소에서 분비되는 여성 호르몬으로, 갱년기가 도래하면 에스트로겐의 분비량이 감소하게 되어 갱년기 증상이 발생합니다. 에스트로겐 치료제는 에스트로겐 부족을 보완하여 갱년기 증상을 완화시키는데 효과적입니다.\n \n 2. 프로게스테론 치료제: 프로게스테론은 난소에서 분비되는 여성 호르몬으로, 갱년기가 도래하면 프로게스테론 분비가 감소하게 되어 갱년기 증상이 발생합니다. 프로게스테론 치료제는 프로게스테론 부족을 보완하여 갱년기 증상을 완화시키는데 효과적입니다.\n \n 3. 합성 호르몬 치료제: 에스트로겐과 프로게스테론을 함께 복용하는 약물로, 갱년기 증상을 완화시키는데 효과적입니다.\n \n 4. 비호르몬 치료제: 갱년기 증상을 완화시키는데 필요한 여성 호르몬을 대체하지 않고, 다른 방식으로 갱년기 증상을 완화시키는 약물입니다. 대표적인 비호르몬 치료제로는 세로토닌 재흡수 억제제(SSRI)와 비소이바레릭산 등이 있습니다.'

In [ ]:
return_answer("요즘 잠을 못 자겠어")

'갱년기 불면증은 갱년기 여성들이 겪는 수면장애로, 수면의 질과 양이 감소하는 것이 특징입니다.\xa0갱년기 불면증의 주요 증상은 밤에 자주 깨어나고 잠들기가 어렵고, 잠이 너무 얕아서 일어나면 피로감이 남아있을 수 있으며, 잠을 자면서도 불쾌감과 긴장감이 남아 불안감을 느낄 수도 있습니다.'

In [ ]:
return_answer("기분이 좋았다가 안 좋았다가 감정이 요동쳐")

'기분이 오락가락하는 것은 갱년기의 일부 증상 중 하나일 수 있습니다. 갱년기는 여성의 난소 기능이 감소하면서 일어나는 생리기능의 변화로 인해 호르몬 수준이 변동적으로 변하게 되어 여러 가지 증상이 나타날 수 있습니다. 이 중에서도 기분이 변화하는 증상은 갱년기의 일부 증상 중 하나로 알려져 있습니다. 이러한 증상은 감정적인 변화, 불안감, 우울증 등으로 나타날 수 있습니다.\n그러나 이러한 증상이 갱년기 때문인지 정확하게 판단하기 위해서는 전문가의 진단이 필요합니다. 따라서, 증상이 계속되거나 심해진다면 전문가와 상담하시는 것이 좋습니다.'

In [ ]:
return_answer("갱년기를 극복할 수 있는 방법이 있을까?")

'갱년기 증상을 완화하기 위한 방법은 다양합니다. 아래는 일반적으로 권장되는 방법들입니다.\n\n호르몬 치료: 에스트로겐 또는 프로게스테론을 보충하여 갱년기 증상을 완화하는 방법입니다. 하지만 호르몬 치료는 부작용이 있을 수 있으므로, 전문의와 상의하고 치료를 받는 것이 좋습니다.\n\n생활습관 개선: 건강한 생활습관을 유지하는 것이 중요합니다. 충분한 수면과 휴식을 취하고, 금연 및 음주를 자제하며, 스트레스를 관리하는 것이 도움이 됩니다.\n\n식이요법: 갱년기 때는 식이요법도 중요합니다. 칼슘과 비타민 D가 풍부한 식품을 먹는 것이 좋습니다. 또한 식이섬유가 풍부한 채소와 과일, 단백질을 충분히 섭취하는 것도 중요합니다.\n\n운동: 운동은 갱년기 증상을 완화하는데 도움이 됩니다. 꾸준한 유산소 운동과 근력운동을 하는 것이 좋습니다.\n\n치료적 요법: 갱년기 증상 중 호흡곤란, 땀이 많이 나는 것, 가슴 통증 등 일부 증상은 치료적 요법으로 완화될 수 있습니다. 따라서 전문의와 상의하여 적절한 치료를 받는 것이 좋습니다.\n\n보조 요법: 갱년기 증상을 완화하기 위해 천연 보조 요법도 사용될 수 있습니다. 예를 들어, 대추 추출물, 대황 추출물 등의 보조 요법이 있습니다. 그러나 보조 요법은 전문의와 상의하고 사용하는 것이 좋습니다.\n\n이러한 방법들은 각각의 개인에 따라 효과가 다를 수 있으므로, 각 개인의 상황에 맞게 선택하고 조절하는 것이 중요합니다. 또한 갱년기 증상이 심각한 경우, 전문의와 상의하고 치료를 받는 것이 좋습니다.'

In [ ]:
return_answer("으라차차")

'푸시업은 가슴과 삼두근을 강화하는 운동입니다. 아래와 같이 진행합니다.\n \n 바닥에 엎드립니다.\n 양손을 어깨 너비로 벌립니다.\n 팔꿈치를 굽혀 몸을 내립니다.\n 팔꿈치를 펴서 몸을 올립니다.\n 플랭크는 복근과 전신 근육을 강화하는 운동입니다. 아래와 같이 진행합니다.\n \n 바닥에 엎드립니다.\n 팔꿈치를 굽혀 팔꿈치와 발끝만 바닥에 닿게 합니다.\n 몸을 일직선으로 유지합니다.\n 아래 링크에서 자세한 설명과 함께 동영상으로 확인하실 수 있습니다. :'

In [ ]:
texts = [{"role" : "user", "content": ""},]

while True:
  text = input('유저: ')
  embedding = model.encode(text)
  train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
  print(train_data['score'].idxmax()) # score가 가장 높은 행 번호 출력
  answer = train_data.loc[train_data['score'].idxmax()]['A'] # score 가장 높은 행의 Answer 리턴
  print("highest :")
  print(train_data.loc[train_data['score'].idxmax()]['score']) # 가장 큰 cos_sim 수치를 나타냄. = high_dist

  print(f"general chatbot : {answer}")
  print(text)
  high_dist = train_data.loc[train_data['score'].idxmax()]['score']

  texts.append({"role":"user", "content": text},)

  print(texts)

  if high_dist <= 0.64 :
      chat = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", messages=texts
        )
      reply = chat.choices[0].message.content
      print(f"Repretty chatbot : {reply}")
      texts.append({"role": "assistant", "content" : reply})

  else :
    print(f"Repretty chatbot : {answer}")


KeyboardInterrupt: ignored

In [ ]:
# 유저: 남편이랑 사이가 너무 안 좋아
# 236
# highest :
# 0.6225272
# Repretty chatbot : 갱년기 불면증에 효과적인 방법은 크게 3가지로 나눌 수 있습니다. 첫째, 생활 습관 개선으로 규칙적인 생활 패턴, 건강한 식습관, 운동, 스트레스 관리 등을 실천하는 것이 중요합니다. 둘째, 수면 환경 개선으로 적절한 온도와 조용한 환경을 조성하고, 침실에서 스마트폰이나 TV 시청을 자제하는 것이 좋습니다. 셋째, 전문가 상담과 약물 치료가 있습니다. 갱년기 불면증은 정신적, 신체적인 문제가 함께 작용하여 발생하기 때문에, 전문가 상담을 받으면 치료 및 조언을 받을 수 있습니다. 또한, 약물 치료를 통해 증상을 완화하는 것도 방법 중 하나입니다. 이러한 방법들을 함께 시도해보면, 갱년기 불면증을 완화할 수 있으며, 더 나은 수면을 취할 수 있습니다.


# 유저: 휴식을 취하는 방법
# 426
# highest :
# 0.68028075
# Repretty chatbot : 몸과 마음을 편안하게 하는 방법은 다양합니다. 스트레칭을 하거나 운동을 하면 몸의 근육을 늘리고 이완시켜서 혈액순환을 원활하게 하여 피로감을 줄이는 데 매우 효과적입니다1. 호흡에 집중하거나 명상을 하면 마음의 안정을 취할 수 있습니다2. 일기를 쓰거나 그림을 그리는 것도 마음의 안정에 도움이 됩니다2. 또한, 아로마 테라피를 이용하거나 차를 마시는 것도 좋은 방법입니다3. 이 외에도 다양한 방법이 있으니 자신에게 맞는 방법을 찾아보시는 것도 좋을 것 같습니다.



# 현재 high_dist 기준점을 0.6으로 두고 있음.

# 1번, 2번 모두 0.6 이상이라 DB based 답변임.
# 근데 1번 질문에 대한 답변은 어색한 것을 볼 수 있음. (high_dist : 0.62정도)

# 2번 질문에 대한 답은 자연스러운 것 같음. (high_dist : 0.68)

# => 기준점을 0.64로 조정함.


In [ ]:
#ChatGPT 주제 유사한지?
def isSimilar(sentence, subject) :
  chat = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", messages=[{"role": "system", "content": 'assistant는 갱년기에 관한 상담을 해주는 상담가이다'}, {"role" : "user", "content" : "'{0}'라는 질문과 {1}가 어느정도 관련성이 있을경우 'true'라고만 답해주고 그렇지 않거나 판단할 수 없다면 'false'라고만 답해줘".format(sentence, subject)}]
        )
  reply = chat.choices[0].message.content
  print(f"Repretty chatbot : {reply}")

In [ ]:
isSimilar("너무 우울하네요", "갱년기")

Repretty chatbot : false


In [ ]:
isSimilar("나 혼자 있고 싶어", "갱년기")

Repretty chatbot : false


In [ ]:
isSimilar("으라차차차차", "갱년기")

Repretty chatbot : false


In [ ]:
isSimilar("행복한 하루 되세요", "갱년기")

RateLimitError: ignored

In [ ]:
isSimilar("지금 몇시야?", "갱년기")

Repretty chatbot : false


In [ ]:
# 주제가 갱년기와 관련이 있는지 판단하는 함수

def isSimilar(question):
  theme = [{"role" : "user", "content": ""},]
  question = "\"" + question + "\"라는 말은 갱년기와 관련이 있을까? 조금이라도 관련있으면 True, 그게 아니라면 False로만 대답해줘." #질문을 조금만 바꿔도 값이 달라질 수 있으니 유의
  # print(question)
  theme.append({"role":"user", "content": question},)

  # print(theme)
  chat = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", messages=theme
        )
  reply = chat.choices[0].message.content
  # print(f"Repretty chatbot : {reply}")

  if(reply == "True"):
    return 1
  else :
    return 0

In [ ]:
num = isSimilar("지금 몇시야?")
num

0

In [ ]:
isSimilar("너무 우울하네요")

1

In [ ]:
isSimilar("나 혼자 있고 싶어")

0

In [ ]:
texts = [{"role" : "user", "content": ""},]

while True:
  text = input('유저: ')
  embedding = model.encode(text)
  train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
  print(train_data['score'].idxmax()) # score가 가장 높은 행 번호 출력
  answer = train_data.loc[train_data['score'].idxmax()]['A'] # score 가장 높은 행의 Answer 리턴
  print("highest :")
  print(train_data.loc[train_data['score'].idxmax()]['score']) # 가장 큰 cos_sim 수치를 나타냄. = high_dist

  print(f"general chatbot : {answer}")
  high_dist = train_data.loc[train_data['score'].idxmax()]['score']

  texts.append({"role":"user", "content": text},)

  print(texts)
  questions = [{"role" : "user", "content": ""},]
  question = "\"" + text + "\"에 후행될만한 질문을 갱년기와 관련해서 3개 예측해줘"
  questions.append({"role":"user", "content": question},)

  if isSimilar(text)==1 :
      chat = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", messages=questions
        )
      reply = chat.choices[0].message.content
      print(f"예측 질문 : {reply}")

  if high_dist <= 0.64 :
      chat = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", messages=texts
        )
      reply = chat.choices[0].message.content
      print(f"Repretty chatbot : {reply}")
      texts.append({"role": "assistant", "content" : reply})

  else :
    print(f"Repretty chatbot : {answer}")


# 관련있을 때 질문 3가지 받는 것까지 구현 완료
# 질문 refining, tokenizing 안 했음.
# 1. ~ 2. ~ 3. ~ 여기서 질문만 따로 이차원 배열에 저장하는 코드 짜야함.
# 질문 고르거나 입력하는 코드 짜야함.
# 아악! 너무 많아.

147
highest :
0.5366664
general chatbot : 기분이 오락가락하는 것은 갱년기의 일부 증상 중 하나일 수 있습니다. 갱년기는 여성의 난소 기능이 감소하면서 일어나는 생리기능의 변화로 인해 호르몬 수준이 변동적으로 변하게 되어 여러 가지 증상이 나타날 수 있습니다. 이 중에서도 기분이 변화하는 증상은 갱년기의 일부 증상 중 하나로 알려져 있습니다. 이러한 증상은 감정적인 변화, 불안감, 우울증 등으로 나타날 수 있습니다.
그러나 이러한 증상이 갱년기 때문인지 정확하게 판단하기 위해서는 전문가의 진단이 필요합니다. 따라서, 증상이 계속되거나 심해진다면 전문가와 상담하시는 것이 좋습니다.
[{'role': 'user', 'content': ''}, {'role': 'user', 'content': '나 요새 좀 우울해'}]
예측 질문 : 1. 갱년기 동안 우울감을 느끼는 이유가 무엇인가요?
2. 갱년기 중에 어떤 신체적이나 정서적 변화가 있는지 알려주세요.
3. 갱년기 우울증을 완화하기 위해 어떤 자기 도움 방법이 있을까요?
Repretty chatbot : 그렇군요. 우울한 기분은 모두가 한번쯤 경험하는 것이라 이해해 드립니다. 우울한 기분이 지속된다면 어떤 문제가 있는지 파악하고 지원을 받아야 할 수도 있습니다. 하지만 일시적인 우울한 기분을 극복하기 위해 몇 가지 방법을 제안해 드릴까요?
